# Datawrapper Tutorial

Datawrapper ist ein service um ansprechende, informative und einfach zu nutzende Visualisierungen zu erstellen. In diesem Tutorial werden einige Beispiele gezeigt um Visualisierungen mit Datawrapper zu erstellen.

Für dieses Tutorial werden die Python Pakete `datawrapper`, `requests` und `pandas` benötigt, welche mit `pip` installiert werden können.

In [1]:
!pip --quiet install datawrapper requests pandas

Nun können Daten von der OpenLigaDB zu den Spieltagsergebnissen der Bundesliga Saison 2020/21 abgerufen werden und in einem DataFrame gespeichert werden.

In [2]:
import requests
import pandas as pd

antwort = requests.get('https://www.openligadb.de/api/getmatchdata/bl1/2020')
spiele = pd.DataFrame(antwort.json())

Das JSON Datenformat von OpenLigaDB enthält einige verschachtelte Datenstrukturen (z.B. Group, Team1, Team2) welche für ein Tabellenformat wie DataFrame nicht gut geeignet sind. Mittels der Funktion `head(n)` können die ersten `n` Werte des DataFrame ausgegeben werden um das Problem zu sehen.

In [3]:
spiele.head(1)

,MatchID,MatchDateTime,TimeZoneID,LeagueId,LeagueName,MatchDateTimeUTC,Group,Team1,Team2,LastUpdateDateTime,MatchIsFinished,MatchResults,Goals,Location,NumberOfViewers
0,58577,2020-09-18T20:30:00,W. Europe Standard Time,4442,1. Fußball-Bundesliga 2020/2021,2020-09-18T18:30:00Z,"{'GroupName': '1. Spieltag', 'GroupOrderID': 1...","{'TeamId': 40, 'TeamName': 'FC Bayern München'...","{'TeamId': 9, 'TeamName': 'FC Schalke 04', 'Sh...",2020-09-20T16:15:35.617,True,"[{'ResultID': 92144, 'ResultName': 'Endergebni...","[{'GoalID': 84638, 'ScoreTeam1': 1, 'ScoreTeam...","{'LocationID': 34, 'LocationCity': 'München', ...",NaN


Die Daten können wie folgt entpackt werden. Zum einen wird die DataFrame Methode `explode()` genutzt, wodurch für Listen in einer verschachtelten Datenstruktur zusätzliche Zeilen pro Eintrag erstellt werden, bei denen alle Werte gleich sind außer denen aus der verschachtelten (explodierten) Datenstruktur. Danach wird der Dataframe wieder in ein JSON Format umgewandet und dieses mittels der DataFrame Methode `json_normalize()` importiert. Hierdurch werden für jeden Eintrag in verschachtelten Datenstrukturen eine eigene Spalte erstellt, wobei sich der Name der Spalte aus dem Namen der ursprünglichen Spalte und dem Namen des Schlüssels innerhalb der Spalte zusammensetzt (z.B. `Team1.TeamName`).

In [4]:
import json
json = json.loads(spiele.explode('MatchResults').explode('Goals').to_json(orient="records"))    
spiele = pd.json_normalize(json)

Die resultierenden Spaltennamen können wie folgt ausgegeben werden:

In [5]:
list(spiele.columns.values)

['MatchID',
 'MatchDateTime',
 'TimeZoneID',
 'LeagueId',
 'LeagueName',
 'MatchDateTimeUTC',
 'LastUpdateDateTime',
 'MatchIsFinished',
 'NumberOfViewers',
 'Group.GroupName',
 'Group.GroupOrderID',
 'Group.GroupID',
 'Team1.TeamId',
 'Team1.TeamName',
 'Team1.ShortName',
 'Team1.TeamIconUrl',
 'Team1.TeamGroupName',
 'Team2.TeamId',
 'Team2.TeamName',
 'Team2.ShortName',
 'Team2.TeamIconUrl',
 'Team2.TeamGroupName',
 'MatchResults.ResultID',
 'MatchResults.ResultName',
 'MatchResults.PointsTeam1',
 'MatchResults.PointsTeam2',
 'MatchResults.ResultOrderID',
 'MatchResults.ResultTypeID',
 'MatchResults.ResultDescription',
 'Goals.GoalID',
 'Goals.ScoreTeam1',
 'Goals.ScoreTeam2',
 'Goals.MatchMinute',
 'Goals.GoalGetterID',
 'Goals.GoalGetterName',
 'Goals.IsPenalty',
 'Goals.IsOwnGoal',
 'Goals.IsOvertime',
 'Goals.Comment',
 'Location.LocationID',
 'Location.LocationCity',
 'Location.LocationStadium',
 'Goals',
 'Location',
 'MatchResults']

Als nächstes werden wir visualisieren, in welchen Zeitintervallen wie viele Tore fallen (z.B. um zu sehen ob vor der Halbzeitpause besonders viele Tore fallen). Hierzu extrahieren wir zunächst die Information in welcher Minute ein Tor gefallen ist (`Goals.MatchMinute`) sowie die eindeutige Referenz eines jeden Tores (`Goals.GoalID`). Wie vorher beschrieben wurden viele Zeilen wiederholt, so dass es mehrere Zeilen pro Tor gibt. Mit der Funktion `drop_duplicates()` erhalten wir die eindeutigen Tore anhand der Tor-Referenz. Danach extrahieren wir noch `Goals.MatchMinute` und erhalten eine Series.

In [6]:
torverteilung = spiele[['Goals.MatchMinute','Goals.GoalID']].drop_duplicates()['Goals.MatchMinute']

Als nächstes erstellen wir ein Histogramm der Tore in verschiedenen Zeitintervallen. Hierfür kann die Pandas Methode `cut()` verwendet werden, welche auch eine automatische Einteilung in Intervalle vornehmen kann. Wir möchten jedoch 5-Minuten Intervalle festlegen bis zur 90. Minuten sowie ein zusätzliches Intervall ab der 90. Minute. Die eigenen Intervalle können mit dem Parameter `bins` definiert werden. Hierbei wird zunächst mittels der `range` Funktion eine Liste von Zahlen von 0 bis 90 (range ist immer exklusive des letzten Wertes) erzeugt. Das Ergebnis wird mit Hilfe des Star Operator als einzelne Werte ausgegeben und mit dem letzten Wert `infinity` kombiniert. Abschließend werden mittels `value_counts()` die Anzahl Tore in den einzelnen Intervallen ausgegeben, wobei diese nicht nach der Anzahl der Tore sortiert werden sollen (`sort=False`).

In [7]:
torverteilung = pd.cut(torverteilung,bins=[*range(0,95,5),float("inf")]).value_counts(sort=False)

Im nächsten Schritt wird noch der Index der Serie in `Zeitintervall` umbenannt und der Name der Series in `Anzahl Tore`. Hierdurch kann die Series direkt zur Visualisierung genutzt werden.

In [8]:
torverteilung = torverteilung.rename_axis("Zeitintervall", axis=0).rename("Anzahl Tore")
torverteilung = torverteilung.to_frame().reset_index()

Das Histogramm kann nun z.B. als CSV ausgegeben werden:

In [9]:
print(torverteilung.to_csv())

,Zeitintervall,Anzahl Tore
0,"(0.0, 5.0]",20
1,"(5.0, 10.0]",35
2,"(10.0, 15.0]",24
3,"(15.0, 20.0]",27
4,"(20.0, 25.0]",28
5,"(25.0, 30.0]",44
6,"(30.0, 35.0]",39
7,"(35.0, 40.0]",30
8,"(40.0, 45.0]",37
9,"(45.0, 50.0]",46
10,"(50.0, 55.0]",39
11,"(55.0, 60.0]",37
12,"(60.0, 65.0]",38
13,"(65.0, 70.0]",34
14,"(70.0, 75.0]",31
15,"(75.0, 80.0]",49
16,"(80.0, 85.0]",27
17,"(85.0, 90.0]",35
18,"(90.0, inf]",32



Nun werden die Daten mit [Datawrapper](datawrapper.de) visualisiert. Für die Verbindung mit Datawrapper wird ein API Access Token benötigt, welcher auf der Webseite unter Settings->API-Tokens erstellt werden kann.

In [ ]:
from getpass import getpass
access_token = getpass("Bitte Access Token eingeben: ") 

Mit dem Access Token kann nun eine Verbindung zu Datawrapper hergestellt werden.

In [ ]:
from datawrapper import Datawrapper
dw = Datawrapper(access_token = access_token)

Nun kann eine Visualisierung der Daten erstellt werden. Die möglichen [Chart Types können der Datawrapper API Dokumentation](https://developer.datawrapper.de/docs/chart-types) entnommen werden.

In [ ]:
torverteilung_chart = dw.create_chart(title = "Torverteilung (Geschossene Tore pro Zeitintervall)", chart_type = 'd3-bars', data = torverteilung)

Die erstellte Visualisierung kann mit `export_chart` heruntergeladen und angezeigt werden.

In [ ]:
dw.export_chart(torverteilung_chart['id'],output = 'png', filepath = 'chart.png', display = True)

Abschließend kann die Visualisierung im Internet öffentlich bereitgestellt werden mit:

In [ ]:
dw.publish_chart(torverteilung_chart['id'])

Die URL kann abgerufen werden mit

In [ ]:
dw.get_iframe_code(torverteilung_chart['id'])